In [2]:
#| label: R packages
#| echo: true
#| code-fold: true
library(tidyverse, quietly=TRUE)
library(lme4)
library(emmeans)
library(DHARMa)
library('readxl')
library('xlsx')
library(dplyr)

task_summary <- read.csv("U:/Documents/Disgust learning project/github/disgust_reversal_learning-final/csvs/dem_vids_task_excluded.csv")

<p>Firstly, add columns to the data-frame splitting conditions into 'disgust or not' (disgust vs fear and points) and 'emotion or not' (points vs disgust and fear)</p>

In [3]:
task_summary <- task_summary %>%
  mutate(
    disgustOrNot = ifelse(block_type == "Disgust", "Disgust", "Not"),
    emotionOrNot = ifelse(block_type == "Points", "Not", "Emotion")
  )

<p>Run the hypothesis test for 'disgust or not' (using the same model specification as used for the hypothesis testing analysis).</p>
<p>In this case, a generalized mixed effects model with:</p>
<p>
* Gamma probability distribution and inverse link function
* no additional random effects or slopes
* no additional covariates
</p>

In [4]:
task_summary$pos_perseverative_er <- task_summary$mean_perseverative_er + 0.01 ##+0.01 as all values must be positive (i.e., can't have 0s)
disgustOrNot <- glmer(pos_perseverative_er ~ disgustOrNot + (1|participant_no), data=task_summary, family=Gamma(link="inverse"))
summary(disgustOrNot)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( inverse )
Formula: pos_perseverative_er ~ disgustOrNot + (1 | participant_no)
   Data: task_summary

     AIC      BIC   logLik deviance df.resid 
  1585.7   1605.4   -788.9   1577.7     1016 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.2631 -0.7119 -0.2032  0.5198  3.9830 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_no (Intercept) 0.1445   0.3801  
 Residual                   0.6149   0.7841  
Number of obs: 1020, groups:  participant_no, 340

Fixed effects:
                Estimate Std. Error t value Pr(>|z|)    
(Intercept)      1.32188    0.07276  18.167   <2e-16 ***
disgustOrNotNot  0.16036    0.06799   2.358   0.0184 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
dsgstOrNtNt -0.597

<p>And then again for 'emotion or 'not':<p>

In [5]:
task_summary$pos_perseverative_er <- task_summary$mean_perseverative_er + 0.01 ##+0.01 as all values must be positive (i.e., can't have 0s)
emotionOrNot <- glmer(pos_perseverative_er ~ emotionOrNot + (1|participant_no), data=task_summary, family=Gamma(link="inverse"))
summary(emotionOrNot)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( inverse )
Formula: pos_perseverative_er ~ emotionOrNot + (1 | participant_no)
   Data: task_summary

     AIC      BIC   logLik deviance df.resid 
  1588.2   1607.9   -790.1   1580.2     1016 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.2593 -0.7077 -0.1818  0.4985  3.8923 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_no (Intercept) 0.1460   0.3821  
 Residual                   0.6178   0.7860  
Number of obs: 1020, groups:  participant_no, 340

Fixed effects:
                Estimate Std. Error t value Pr(>|z|)    
(Intercept)      1.38443    0.06292  22.001   <2e-16 ***
emotionOrNotNot  0.12797    0.07535   1.698   0.0894 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation of Fixed Effects:
            (Intr)
emotnOrNtNt -0.371

<p>Only the 'disgust or not' model finds a significant effect, suggesting that the difference between disgust and other types of learning (both fear and points learning) is the key driver of effects<p>
<p>BIC values also show that this model is the better fitting model, further supporting the hypothesis</p>

In [6]:
bic_values <- c(
    BIC(disgustOrNot),
    BIC(emotionOrNot)
)
model_names <- c("Disgust or not", "Emotion or not")
bic_df <- data.frame(Model = model_names, BIC = bic_values)
bic_df <- bic_df[order(bic_df$BIC), ]

print(bic_df)

           Model      BIC
1 Disgust or not 1605.429
2 Emotion or not 1607.863
